In [1]:
import pygame
import sys
import random

pygame 2.6.1 (SDL 2.28.4, Python 3.12.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
SCREEN_WIDTH, SCREEN_HEIGHT = 800, 600

In [3]:
class Character:
    def __init__(self, x, y, size, image_paths, screen):
        self.x = x
        self.y = y
        self.width = size[0]
        self.height = size[1]
        self.screen = screen
        self.speed = 5
        self.jump_power = 15
        self.is_jumping = False
        self.jump_height = 0
        self.image_paths = image_paths 
        self.current_image_index = 0  
        self.image = pygame.image.load(self.image_paths[self.current_image_index])
        self.image = pygame.transform.scale(self.image, size)

    def draw(self):
        self.screen.blit(self.image, (self.x, self.y))

    def move(self, left=False, right=False):
        if left:
            if self.x - self.speed >= 0:
                self.x -= self.speed
                self.update_animation()  
        if right:
            if self.x + self.width + self.speed <= SCREEN_WIDTH:
                self.x += self.speed
                self.update_animation()  

    def jump(self):
        if not self.is_jumping:
            self.is_jumping = True
            self.jump_height = self.jump_power

    def update(self):
        if self.is_jumping:
            self.y -= self.jump_height
            self.jump_height -= 0.7
            if self.y >= 360: 
                self.y = 360
                self.is_jumping = False
                self.jump_height = 0

    def update_animation(self):
        self.current_image_index += 1
        if self.current_image_index >= len(self.image_paths):
            self.current_image_index = 0 
        
        self.image = pygame.image.load(self.image_paths[self.current_image_index])
        self.image = pygame.transform.scale(self.image, (self.width, self.height))
        
    def get_mask(self):
        return pygame.mask.from_surface(self.image)
    

In [4]:
class Obstacles:
    def __init__(self, screen, images_with_sizes, start_x, ground_y, speed):
        self.screen = screen
        self.images_with_sizes = images_with_sizes
        self.positions = [] 
        self.start_x = start_x  
        self.ground_y = ground_y  
        self.speed = speed 
        self.spawn_obstacles()

    def spawn_obstacles(self):
        x = self.start_x
        for _ in range(5): 
            obstacle_data = random.choice(self.images_with_sizes) 
            obstacle_image = pygame.image.load(obstacle_data[0])
            obstacle_image = pygame.transform.scale(obstacle_image, obstacle_data[1])
            self.positions.append({
                "image": obstacle_image,
                "x": x,
                "y": self.ground_y - obstacle_image.get_height(),
                "mask": pygame.mask.from_surface(obstacle_image)
            })
            x += random.randint(300, 500)

    def update(self):
        for obstacle in self.positions:
            obstacle["x"] -= self.speed 

        self.positions = [
            obs for obs in self.positions if obs["x"] + obs["image"].get_width() > 0
        ]
        if len(self.positions) < 5: 
            last_x = max(obs["x"] for obs in self.positions)
            obstacle_data = random.choice(self.images_with_sizes)
            new_obstacle_image = pygame.image.load(obstacle_data[0])
            new_obstacle_image = pygame.transform.scale(new_obstacle_image, obstacle_data[1])
            self.positions.append({
                "image": new_obstacle_image,
                "x": last_x + random.randint(300, 500),
                "y": self.ground_y - new_obstacle_image.get_height(),
                "mask": pygame.mask.from_surface(new_obstacle_image)
            })

    def draw(self):
        for obstacle in self.positions:
            self.screen.blit(obstacle["image"], (obstacle["x"], obstacle["y"]))

    def check_collision(self, knight):
        knight_mask = knight.get_mask()
        knight_offset_x = knight.x
        knight_offset_y = knight.y

        for obstacle in self.positions:
            offset = (obstacle["x"] - knight_offset_x, obstacle["y"] - knight_offset_y)
            if knight_mask.overlap(obstacle["mask"], offset):
                return True
        return False


In [ ]:
pygame.init()

SCREEN_WIDTH, SCREEN_HEIGHT = 800, 600
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("ENDLESS RUNNER")

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)
RED = (225, 0, 0)

# CZCIONKI
title_font = pygame.font.Font("April Signature.otf", 80) 
font = pygame.font.Font(None, 74) 
small_font = pygame.font.Font(None, 36)

# TŁO
background_image = pygame.image.load("background.png")
background_image = pygame.transform.scale(background_image, (SCREEN_WIDTH, SCREEN_HEIGHT))

knight_image = pygame.image.load("Walk (9).png")
knight_image = pygame.transform.scale(knight_image, (380, 370))

play_background_image = pygame.image.load("BG.png")
play_background_image = pygame.transform.scale(play_background_image, (SCREEN_WIDTH, SCREEN_HEIGHT-100))

play_tile_image = pygame.image.load("Tile (2).png")
play_tile_image = pygame.transform.scale(play_tile_image, (400, 220))

# MUZYKA
pygame.mixer.music.load("music.mp3")
pygame.mixer.music.play(-1)

class GameWindow:
    def __init__(self, screen):
        self.screen = screen
        self.mx, self.my = pygame.mouse.get_pos()
    
    def draw_text(self, text, font, color, x, y):
        text_obj = font.render(text, True, color)
        text_rect = text_obj.get_rect(center=(x, y))
        self.screen.blit(text_obj, text_rect)

    def handle_events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        self.mx, self.my = pygame.mouse.get_pos()

    def draw_button(self, button, color, text, font, x, y):
        pygame.draw.rect(self.screen, color, button)
        self.draw_text(text, font, WHITE, x, y)

    def is_mouse_over(self, button):
        return button.collidepoint((self.mx, self.my))
    
def main_menu():
    game_window = GameWindow(screen)
    
    while True:
        screen.blit(background_image, (0, 0))  # Tło
        screen.blit(knight_image, (70, SCREEN_HEIGHT // 2 - 150))  # Rycerz

        game_window.draw_text("KNIGHT", title_font, WHITE, SCREEN_WIDTH // 2, 70)  
        game_window.draw_text("ADVENTURES", title_font, WHITE, SCREEN_WIDTH // 2, 170)

        button_play = pygame.Rect(SCREEN_WIDTH - 300, 250, 200, 50)
        button_rules = pygame.Rect(SCREEN_WIDTH - 300, 320, 200, 50)
        button_about = pygame.Rect(SCREEN_WIDTH - 300, 390, 200, 50)
        button_exit = pygame.Rect(SCREEN_WIDTH - 300, 460, 200, 50)

        # Play button
        if game_window.is_mouse_over(button_play):
            game_window.draw_button(button_play, GRAY, "Play", font, SCREEN_WIDTH - 200, 275)
            if pygame.mouse.get_pressed()[0]:
                play_game()
        else:
            game_window.draw_button(button_play, BLACK, "Play", font, SCREEN_WIDTH - 200, 275)

        # Rules button
        if game_window.is_mouse_over(button_rules):
            game_window.draw_button(button_rules, GRAY, "Rules", font, SCREEN_WIDTH - 200, 345)
            if pygame.mouse.get_pressed()[0]:
                show_rules()
        else:
            game_window.draw_button(button_rules, BLACK, "Rules", font, SCREEN_WIDTH - 200, 345)

        # About button
        if game_window.is_mouse_over(button_about):
            game_window.draw_button(button_about, GRAY, "About", font, SCREEN_WIDTH - 200, 415)
            if pygame.mouse.get_pressed()[0]:
                show_about()
        else:
            game_window.draw_button(button_about, BLACK, "About", font, SCREEN_WIDTH - 200, 415)

        # Exit button
        if game_window.is_mouse_over(button_exit):
            game_window.draw_button(button_exit, GRAY, "Exit", font, SCREEN_WIDTH - 200, 485)
            if pygame.mouse.get_pressed()[0]:
                pygame.quit()
                sys.exit()
        else:
            game_window.draw_button(button_exit, BLACK, "Exit", font, SCREEN_WIDTH - 200, 485)

        pygame.display.flip()
        game_window.handle_events()

def show_rules():
    game_window = GameWindow(screen)
    
    while True:
        screen.fill(BLACK) 

        game_window.draw_text("Game Rules", title_font, WHITE, SCREEN_WIDTH // 2, 100)
        game_window.draw_text("- Use the arrow keys to move the knight.", small_font, WHITE, SCREEN_WIDTH // 2 - 35, 200)
        game_window.draw_text("- Avoid obstacles and keep running!", small_font, WHITE, SCREEN_WIDTH // 2 - 58, 250)
        game_window.draw_text("- The longer you run, the more points you get.", small_font, WHITE, SCREEN_WIDTH // 2, 300)

        back_button = pygame.Rect(40, 450, 200, 50)
        if game_window.is_mouse_over(back_button):
            game_window.draw_button(back_button, GRAY, "Back", font, 140, 475)
            if pygame.mouse.get_pressed()[0]:
                return
        else:
            game_window.draw_button(back_button, BLACK, "Back", font, 140, 475)

        pygame.display.flip()
        game_window.handle_events()

def show_about():
    game_window = GameWindow(screen)
    
    while True:
        screen.fill(BLACK)

        game_window.draw_text("About the Game", title_font, WHITE, SCREEN_WIDTH // 2, 100)
        game_window.draw_text("Hi, I'm Jagoda!", small_font, WHITE, SCREEN_WIDTH // 2, 200)
        game_window.draw_text("I hope you enjoy the game!", small_font, WHITE, SCREEN_WIDTH // 2, 250)
        game_window.draw_text("Have fun and good luck!", small_font, WHITE, SCREEN_WIDTH // 2, 300)

        back_button = pygame.Rect(40, 450, 200, 50)
        if game_window.is_mouse_over(back_button):
            game_window.draw_button(back_button, GRAY, "Back", font, 140, 475)
            if pygame.mouse.get_pressed()[0]:
                return
        else:
            game_window.draw_button(back_button, BLACK, "Back", font, 140, 475)

        pygame.display.flip()
        game_window.handle_events()

def play_game():
    game_window = GameWindow(screen)
    clock = pygame.time.Clock()
    bg_x = 0

    run_images = [f"Run ({i}).png" for i in range(1, 11)]
    knight = Character(x=30, y=360, size=(110, 130), image_paths=run_images, screen=screen)

    obstacle_images_with_sizes = [
        ("Bush (1).png", (80, 60)),
        ("Bush (2).png", (95, 70)),
        ("Crate.png", (50, 60)),
        ("Skeleton.png", (60, 40))
    ]

    obstacles = Obstacles(
        screen=screen,
        images_with_sizes=obstacle_images_with_sizes,
        start_x=SCREEN_WIDTH,
        ground_y=470,
        speed=5
    )

    start_time = pygame.time.get_ticks()
    last_collision_time = -1000

    lives = 3
    points = 0

    while True:
        if lives <= 0:
            game_over_screen(points) 
            return

        bg_x -= 3
        if bg_x <= -SCREEN_WIDTH:
            bg_x = 0

        screen.blit(play_background_image, (bg_x, 0))
        screen.blit(play_background_image, (bg_x + SCREEN_WIDTH, 0))
        screen.blit(play_tile_image, (0, 470))
        screen.blit(play_tile_image, (400, 470))

        obstacles.update()
        obstacles.draw()

        knight.draw()

        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            knight.move(left=True)
        if keys[pygame.K_RIGHT]:
            knight.move(right=True)
        if keys[pygame.K_UP]:
            knight.jump()

        knight.update()

        # OPÓŹNIENIE KOLEJNEJ KOLIZJI
        current_time = pygame.time.get_ticks()
        if current_time - last_collision_time > 1000:
            if obstacles.check_collision(knight):
                lives -= 1
                knight.x = 30
                knight.y = 360
                last_collision_time = current_time 

        points = (pygame.time.get_ticks() - start_time) / 100
        game_window.draw_text(f"POINTS: {int(points)}", small_font, WHITE, SCREEN_WIDTH - 100, 50)

        # Wyświetlanie liczby żyć
        game_window.draw_text(f"LIVES: {lives}", small_font, RED, 100, 50)

        pygame.display.flip()
        clock.tick(60)
        game_window.handle_events()

def game_over_screen(points):
    game_window = GameWindow(screen)
    
    while True:
        screen.fill(BLACK)  # Czarny ekran

        # Ekran "GAME OVER"
        game_window.draw_text("GAME OVER", title_font, WHITE, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 3)
        game_window.draw_text(f"YOUR SCORE: {int(points)}", small_font, RED, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 )
        game_window.draw_text("DO YOU WANT TO PLAY AGAIN?", small_font, WHITE, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + 40)


        button_yes = pygame.Rect(SCREEN_WIDTH // 2 - 100, SCREEN_HEIGHT // 2 + 80, 200, 50)
        button_no = pygame.Rect(SCREEN_WIDTH // 2 - 100, SCREEN_HEIGHT // 2 + 150, 200, 50)

        if game_window.is_mouse_over(button_yes):
            game_window.draw_button(button_yes, GRAY, "YES", font, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + 105)
            if pygame.mouse.get_pressed()[0]:
                play_game() 
                return
        else:
            game_window.draw_button(button_yes, BLACK, "YES", font, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + 105)

        if game_window.is_mouse_over(button_no):
            game_window.draw_button(button_no, GRAY, "NO", font, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + 175)
            if pygame.mouse.get_pressed()[0]:
                main_menu() 
                return
        else:
            game_window.draw_button(button_no, BLACK, "NO", font, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + 175)

        pygame.display.flip()
        game_window.handle_events()

main_menu()

SystemExit: 

C:\Users\ploci\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
